# Testing merges before turning them into functions

In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata2()
organizations = data_prep.load_cleaned_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Merge NTD with GTFS  --> vehicle_gtfs


In [3]:
#drop records without NTD ID.
#organizations = organizations.dropna(subset=['ntd_id'])

In [4]:
#put in script later
vehicles.ntd_id = vehicles.ntd_id.astype(str)

In [5]:
#put in script later
organizations.ntd_id = organizations.ntd_id.astype(str)

In [6]:
organizations[organizations.name.str.contains("Butte County Association")]["ntd_id"].iloc[0]

'90208'

In [7]:
vehicles[vehicles.agency.str.contains("Butte")]["ntd_id"].iloc[0]

'90208'

In [8]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [9]:
vehicles_gtfs['_merge'].value_counts()

both          646
left_only      59
right_only      0
Name: _merge, dtype: int64

In [10]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

In [11]:
vehicles_gtfs.head(2)

,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups,name,itp_id,gtfs_schedule_status,number_services_w_complete_rt_status,#_fixed_route_services_w__static_gtfs,complete_static_gtfs_coverage_1_means_yes,complete_rt_coverage,gt_1_gtfs_for_any_service_1_means_yes,gt_1_complete_rt_set_1_means_yes
0,Los Angeles County Metropolitan Transportation...,Los Angeles,CA,9154,90154,Independent Public Agency or Authority of Tran...,Full Reporter,12150996,3482,Van,NaN,0,0,0,0,0,0,1208,1.440397,31456.6134,1208,0,Vans,Los Angeles County Metropolitan Transportation...,182.0,"ok,ok,needed",2.0,2.0,1.0,1.0,1.0,1.0
1,Los Angeles County Metropolitan Transportation...,Los Angeles,CA,9154,90154,Independent Public Agency or Authority of Tran...,Full Reporter,12150996,3482,Trucks and other Rubber Tire Vehicles (Service),NaN,39,162,8,5,6,1,961,NaN,NaN,467,273,Vans,Los Angeles County Metropolitan Transportation...,182.0,"ok,ok,needed",2.0,2.0,1.0,1.0,1.0,1.0


# Merging Black Cat with vehicles_gtfs

In [12]:
right_on = ["agency"]
left_on = ["organization_name"]

In [13]:
black_cat1 = fuzzymatcher.fuzzy_left_join(df_5311, vehicles_gtfs, left_on, right_on)

In [14]:
unique_agencies = black_cat1[['best_match_score','organization_name','agency']]
unique_agencies = unique_agencies.drop_duplicates()

In [15]:
unique_agencies

,best_match_score,organization_name,agency
0,0.337941,City of Chowchilla,"City of Chowchilla, dba: Chowchilla Area Transit"
1,0.102801,Madera County,"Madera County, dba: Madera County Connection T..."
9,0.700603,Fresno County Rural Transit Agency,Fresno County Rural Transit Agency
25,0.620698,Yuba-Sutter Transit Authority,Yuba-Sutter Transit Authority
35,0.334219,City of Arcata,City of Arcata
39,0.364322,City of Dinuba,City of Dinuba
41,0.475059,City of Ojai,City of Ojai
47,0.520946,Modoc Transportation Agency,Modoc Transportation Agency
53,0.652190,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System
75,0.420768,Humboldt Transit Authority,Humboldt Transit Authority


In [16]:
#stuff to filter out before cutting off at scores...
subset3 = ['County of Los Angeles - Department of Public Works', 'County of Sacramento Department of Transportation']

In [17]:
#filter out a bunch of stuff all at once
black_cat2 = black_cat1[~black_cat1.organization_name.isin(subset3)]

In [18]:
#filter out for any scores below 0.19
black_cat2 = black_cat2.loc[black_cat2['best_match_score'] > 0.19 ]

In [19]:
#total agencies that matched over with decent scores
black_cat2['organization_name'].nunique()

65

# Walking back in the other 21 agencies that were cut into a seperate dataframe

In [20]:
#getting a list of agencies with bad scores
bad_fuzzy_matches = black_cat1.loc[black_cat1['best_match_score'] < 0.19 ]

In [21]:
bad_fuzzy_matches = bad_fuzzy_matches[['organization_name','agency', 'ntd_id']]

In [22]:
bad_fuzzy_matches = bad_fuzzy_matches.drop_duplicates()

In [23]:
#making a manual crosswalk
#bad_fuzzy_matches.to_csv("./bad_fuzzy_matches.csv", index = False) 

In [24]:
#reading crosswalk 3, what I manually inputted
crosswalk3 = pd.read_excel('corrected_bad_fuzzy_match.xlsx')

In [25]:
#crosswalk on the left, I only want a dataframe with the poorly matched agencies
unmatched = crosswalk3.merge(df_5311, on=['organization_name'],  how='left')

In [26]:
#now grabbing vehicles and GTFS info on the left
unmatched2 = unmatched.merge(vehicles_gtfs, on=['agency','ntd_id'],  how='left')

In [27]:
unmatched2['organization_name'].nunique()

22

# R Bind

In [28]:
GTFS_Vehicles_BC = black_cat2.append(pd.DataFrame(data = unmatched2), ignore_index=True)

In [29]:
GTFS_Vehicles_BC = GTFS_Vehicles_BC.drop(columns=['best_match_score','__id_left','__id_right','vehicle_type'])

In [30]:
GTFS_Vehicles_BC.shape

(925, 46)

## One last check to make sure all the names are there: 26 missing
<b> Still having issues </b>
* Tuolumne County Transit's NTD ID is 9R02-91057 in Cal ITP but 9402-035 in NTD
* Butte

<b> Inherently, aren't sure if these agencies correspond with any records </b> 
* Calaveras Transit Agency,Tuloumne, Alpine County, Fresno Council of Gov, Klamath Trinity Non-Emergency Transportationâ€‹	, Greyhound, County of LA, City of Tehachapi, Calaveras County Work, County of Nevada Public Works, County Connection (Central Contra Costa Transit),County of Shasta Department of Public Works,Glenn County Transportation Commission, Tehama County Transit Agency: not sure if this appears in Cal ITP, no obvious matches found? Erring on the side of cuation. 
* Generate a column that indicates that the agencies above that doesn't have GTFS. Everything else is 0.

In [37]:
Names = GTFS_Vehicles_BC[['organization_name','agency','ntd_id','itp_id','gtfs_schedule_status']]
Names = Names.drop_duplicates()
#Names.sort_values('organization_name')

In [32]:
Names.isna().sum()

organization_name        0
agency                  11
ntd_id                  11
itp_id                  18
gtfs_schedule_status    18
dtype: int64

In [33]:
Names.shape

(87, 5)

## Generate a column that indicates that the rows  that don't have GTFS/NTD ID/Cal ITP, since we aren't able to match these agencies ('true'), while everything else is 'false.'

In [52]:
missing_list = Names[Names.isna().any(axis=1)]

In [55]:
missing_agencies = missing_list[['organization_name']]

In [62]:
missing_agencies=missing_agencies.values.tolist()

In [65]:
type(missing_agencies)

list

In [68]:
GTFS_Vehicles_BC = GTFS_Vehicles_BC.assign(No_GTFS_NTD_Matches = GTFS_Vehicles_BC.apply(lambda x: x.organization_name in missing_agencies, axis=1))

## Final preview
* What to delete - agency, city, state?

In [71]:
GTFS_Vehicles_BC.columns

Index(['grant_fiscal_year', 'funding_program', 'grant_number', 'project_year',
       'organization_name', 'upin', 'description', 'ali', 'contract_number',
       'allocationamount', 'encumbered_amount', 'expendedamount',
       'activebalance', 'closedoutbalance', 'project_status', 'agency', 'city',
       'state', 'legacy_ntd_id', 'ntd_id', 'organization_type',
       'reporter_type', 'primary_uza_population', 'agency_voms', 'years_old:',
       '_13_15', '_16_20', '_21_25', '_26_30', '_31_60', '_60+',
       'total_vehicles', 'average_age_of_fleet__in_years_',
       'average_lifetime_miles_per_vehicle', '_0_9', '_10_12',
       'vehicle_groups', 'name', 'itp_id', 'gtfs_schedule_status',
       'number_services_w_complete_rt_status',
       '#_fixed_route_services_w__static_gtfs',
       'complete_static_gtfs_coverage_1_means_yes', 'complete_rt_coverage',
       'gt_1_gtfs_for_any_service_1_means_yes',
       'gt_1_complete_rt_set_1_means_yes', 'No_GTFS_NTD_Matches'],
      dtype='o

In [69]:
GTFS_Vehicles_BC.head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups,name,itp_id,gtfs_schedule_status,number_services_w_complete_rt_status,#_fixed_route_services_w__static_gtfs,complete_static_gtfs_coverage_1_means_yes,complete_rt_coverage,gt_1_gtfs_for_any_service_1_means_yes,gt_1_complete_rt_set_1_means_yes,No_GTFS_NTD_Matches
0,2011.0,Section 5311,CA-18-X047 | 0012000083,2016.0,City of Chowchilla,BCG0000228,Operating Assistance,300902,64BO17-00368,53221.0,114511.0,53221.0,0.0,0.0,Open,"City of Chowchilla, dba: Chowchilla Area Transit",Chowchilla,CA,9R02-058,9R02-91071,"City, County or Local Government Unit or Depar...",Rural Reporter,0.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.75,0.0,4.0,0.0,Vans,City of Chowchilla,65.0,needed,0.0,0.0,1.0,1.0,0.0,0.0,False


In [72]:
GTFS_Vehicles_BC.shape

(925, 47)

## HELP drop duplicates not working

In [76]:
GTFS_Vehicles_BC = GTFS_Vehicles_BC.drop_duplicates(subset=None, keep="first", inplace=False)

In [77]:
GTFS_Vehicles_BC.shape

(925, 47)

In [70]:
GTFS_Vehicles_BC.to_csv("./GTFS_Vehicles_BC.csv", index = False) 